# Preparing Basis States in PennyLane

## Importing PennyLane and the wrapped version of NumPy

PennyLane is an open source software for quantum machine learning and quantum computing algorithms. It integrates standard machine learning software and tools, such as TensorFlow and Pytorch, with quantum computing software like IBM's Qiskit, Microsoft's Q#, and Google's Cirq. It also has its own built in functions and models, and allows various hardware backends such as the quantum computers available from IBM via free cloud access. PennyLane has a wrapped version of NumPy that we import rather than the standard NumPy we have been using. 

In [1]:
pip install pennylane

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pennylane as qml
from pennylane import numpy as np

## Basis States

In PennyLane, and other quantum computing software, we often want to prepare basis states. This is something we will return to again in the future when we study specific cases of quantum circuits, but having some basic introduction to state preparation now seems appropriate since the are simply tensor products of the basis states $|0\rangle$ and $|1\rangle$. In particular, we can prepare states like

\begin{align}
|00 \rangle &= |0\rangle \otimes |0\rangle = \begin{pmatrix} 1\\0 \end{pmatrix} \otimes \begin{pmatrix} 1\\0 \end{pmatrix} = \begin{pmatrix} 1\\0\\0\\0 \end{pmatrix} \\
|01 \rangle &= |0\rangle \otimes |1\rangle = \begin{pmatrix} 1\\0 \end{pmatrix} \otimes \begin{pmatrix} 0\\1 \end{pmatrix} = \begin{pmatrix} 0\\1\\0\\0 \end{pmatrix} \\
|10 \rangle &= |1\rangle \otimes |0\rangle = \begin{pmatrix} 0\\1 \end{pmatrix} \otimes \begin{pmatrix} 1\\0 \end{pmatrix} = \begin{pmatrix} 0\\0\\1\\0 \end{pmatrix} \\
|11 \rangle &= |1\rangle \otimes |1\rangle = \begin{pmatrix} 0\\1 \end{pmatrix} \otimes \begin{pmatrix} 0\\1 \end{pmatrix} = \begin{pmatrix} 0\\0\\0\\1 \end{pmatrix}
\end{align}

Let's define these in Python

In [6]:
# Define single qubit states spin-up and spin-down
u = np.matrix([[1],
               [0]])
d = np.matrix([[0],
               [1]])

# Define the basis states:
uu = np.kron(u, u)
ud = np.kron(u, d)
du = np.kron(d, u)
dd = np.kron(d, d)

|00> =
[[1]
 [0]
 [0]
 [0]]


In [10]:
print('|00> =')
print(uu)

|00> =
[[1]
 [0]
 [0]
 [0]]


In [7]:
print('|01> =')
print(ud)

|01> =
[[0]
 [1]
 [0]
 [0]]


In [8]:
print('|10> =')
print(du)

|10> =
[[0]
 [0]
 [1]
 [0]]


In [9]:
print('|11> =')
print(dd)

|11> =
[[0]
 [0]
 [0]
 [1]]


Taking this further, we can define states like:
    
\begin{align}
|000\rangle = |0\rangle \otimes |0\rangle \otimes |0\rangle = \begin{pmatrix} 1\\0\\0\\0\\0\\0\\0\\0 \end{pmatrix}
\end{align}

\begin{align}
|010\rangle = |0\rangle \otimes |1\rangle \otimes |0\rangle = \begin{pmatrix} 0\\0\\1\\0\\0\\0\\0\\0 \end{pmatrix}
\end{align}

In PennyLane the following code defines a device (quantum computer or simulator) on which to run code. We will use the default simulator for now since the code is very simple and running it on an actual quantum computer is unnecessary. The number of wires is equal to the number of qubits we are using. The number of shots is the number of times we want to run the circuit.

In [12]:
dev = qml.device("default.qubit", wires=2, shots=1)

Now we can define an array that corresponds to the basis state $|00\rangle$:

In [13]:
uu = np.array([0, 0])

We can define a function `circuit()` that defines a quantum circuit. 

In [18]:
def circuit():
    qml.BasisState(uu, wires=[0, 1])
    return qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))

Now, we define a `qnode` class that runs on the device we have defined and samples the qubits and gives a $+1$ for spin-up and a $-1$ for spin down. This runs a quantum function (circuit) that we have just defined. 

In [19]:
@qml.qnode(dev)
def circuit():
    qml.BasisState(uu, wires=[0, 1])
    return qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))

print(circuit())

[[1]
 [1]]


Since qubits are by default in the spin-up state for any quantum circuit, and since we did not prepare any special initial state or perform any operations on the qubits, we get an expectation value of $+1$ for both when sampling the circuit. The vector above show a $+1$ in the first and second entry meaning it is measuring two spin-up states. Let's define a new state to prepare corresponding to $|01 \rangle$:

In [20]:
ud = np.array([0,1])

Next, let's define a new `qnode` to sample corresponding to this prepared state:

In [21]:
@qml.qnode(dev)
def circuit():
    qml.BasisState(ud, wires=[0, 1])
    return qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))

print(circuit())

[[ 1]
 [-1]]


As we can see, we get a $+1$ in the first component, corresponding to the spin-up state, and we get a $-1$ in the second component corresponding to the spin-down state. We can also tell PennyLane to perform multiple "shots" or samples by defining a new device:

In [22]:
dev2 = qml.device("default.qubit", wires=2, shots=10)

In [23]:
@qml.qnode(dev2)
def circuit():
    qml.BasisState(ud, wires=[0, 1])
    return qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))

print(circuit())

[[ 1  1  1  1  1  1  1  1  1  1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]


This gives us 10 output samples instead of just one. If we define a new array corresponding to the state $|11\rangle$,

In [24]:
dd = np.array([1,1])

We can define a new 'qnode' with the new device we just defined, and we should expect an output vector of 

\begin{align}
\begin{pmatrix}
-1\\-1
\end{pmatrix}
\end{align}

ten times, corresponding to the ten samples:

In [25]:
@qml.qnode(dev2)
def circuit():
    qml.BasisState(dd, wires=[0, 1])
    return qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))

print(circuit())

[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]


We will return to this when we discuss measurements and expectation values. 

## Exercises

### Write code to compute the following basis states:

1. $|000 \rangle$
2. $|001 \rangle$
3. $|010 \rangle$
4. $|011 \rangle$
5. $|100 \rangle$
6. $|101 \rangle$
7. $|110 \rangle$
8. $|111 \rangle$

Now, compute these by hand to verify your code is correct. For example, the first computation will be

\begin{align}
\begin{pmatrix} 1\\0 \end{pmatrix} \otimes
\begin{pmatrix} 1\\0 \end{pmatrix} \otimes
\begin{pmatrix} 0\\1 \end{pmatrix}
\end{align}

9. Define a defaul device with "3 wires" and "5 shots".
10. Define an array 'uud' for the basis state $|001\rangle$.
11. Now define a `qnode` that will print the samples: 

> `qml.sample(qml.PauliZ(0))`

> `qml.sample(qml.PauliZ(1))` 

> `qml.sample(qml.PauliZ(2))`

12. Run your 'qnode' and verify that you get the output vector
\begin{align}
\begin{pmatrix}
1 \\ 1 \\ -1
\end{pmatrix}
\end{align}

five times.

13. Try doing this with the other basis states given in Exercises 1-8.